# L4: Support Data Insight Analysis

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [9]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Loading Tasks and Agents YAML files

In [10]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Using FileReadTool

In [11]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

## Creating Agents, Tasks and Crew

In [12]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


2025-01-14 12:01:44,952 - 139722255448960 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Testing our Crew

In [13]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-01-14 12:01:45,167 - 139722255448960 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\", \"return_direct\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\"}"
## T



# Agent: Report Generator
## Final Answer: 
### Table 1: Issue Classification Results

| Issue Type       | Frequency | Priority Levels | High | Medium | Low | Critical |
|------------------|-----------|------------------|------|--------|-----|----------|
| API Issue        | 8         | 3                | 5    | 1      | 1   | 2        |
| Login Issue      | 7         | 3                | 3    | 1      | 3   | 1        |
| Report Generation | 6         | 3                | 2    | 1      | 3   | 0        |
| Data Import      | 10        | 3                | 5    | 3      | 2   | 0        |
| Feature Request   | 6         | 3                | 4    | 2      | 0   | 0        |
| UI Bug           | 7         | 3                | 3    | 2      | 2   | 1        |
| Billing Issue    | 10        | 3                | 5    | 4      | 1   | 1        |

### Table 2: Agent Performance

| Agent ID | Average Resolution Time (Minutes) | Average Satisfaction Score |
|----------|----------------------

# Agent: Report Generator
## Task: Assemble the final report by integrating the tables and charts generated in the previous tasks. The report should include: - Issue Classification Results: Overview of the types of issues reported,
  their frequency, and priority levels, presented in tables and charts.
- Agent Performance: Insights into the performance of support agents, based
  on metrics like resolution time and customer satisfaction, presented in
  tables and charts.
- Customer Satisfaction: A summary of the customer satisfaction ratings and
  their trends over time, presented in tables and charts.
- Suggested Actions: A summary of the actionable suggestions generated for
  each category of issue.

The report should have tables and relevant charts side by side with an appropriate size.
The report should be formatted for easy consumption by stakeholders, providing valuable insights into the performance of the support system and areas for improvement.



# Agent: Report Generator
## T



# Agent: Report Generator
## Final Answer: 
### Final Report on Support Tickets

#### 1. Issue Classification Results
This section provides an overview of the types of issues reported, their frequency, and priority levels.

| Issue Type       | Frequency | Priority Levels | High | Medium | Low | Critical |
|------------------|-----------|------------------|------|--------|-----|----------|
| API Issue        | 8         | 3                | 5    | 1      | 1   | 2        |
| Login Issue      | 7         | 3                | 3    | 1      | 3   | 1        |
| Report Generation | 6         | 3                | 2    | 1      | 3   | 0        |
| Data Import      | 10        | 3                | 5    | 3      | 2   | 0        |
| Feature Request   | 6         | 3                | 4    | 2      | 0   | 0        |
| UI Bug           | 7         | 3                | 3    | 2      | 2   | 1        |
| Billing Issue    | 10        | 3                | 5    | 4      | 1   | 1        |

#### 2.

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  9.0  │    9.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  9.0  │    9.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  8.0  │    8.0     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  9.3  │    9.3     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.82  │    8.8     │                     │  │  ┃
┃ Execution Time (s) │   1   │     1      │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

## Training your crew and agents

In [14]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

2025-01-14 12:01:45,960 - 139722255448960 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\", \"return_direct\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\"}"
## T


 

Action 'None' don't exist, these are the only available Actions:
Tool Name: Read a file's content(**kwargs: Any) -> Any
Tool Description: Read a file's content() - A tool that can be used to read ./support_tickets_data.csv's content. 
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annotation=type[BaseModel], required=True, description='The tool schema.', metadata=[SkipValidation()]), 'return_direct': FieldInfo(annotation=bool, required=False, default=False), 'verbose': FieldInfo(annotation=bool, required=False, default=False), 'callbacks': FieldInfo(annotation=Union[list[BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None, exclude=True), 'callback_manager': FieldInfo(annotation=Union[BaseCallbackManager, NoneType], required=False, default=None, description='.. deprecated:: 0.1.7 Use :meth:`~callbacks` instead.\n\nCallback manager to add t




# Agent: Report Generator
## Final Answer: 
### Issue Classification Results
| Issue Type         | Frequency | Priority Levels |
|---------------------|-----------|------------------|
| API Issue           | 10        | High: 5, Low: 2, Medium: 2, Critical: 1 |
| Login Issue         | 9         | High: 4, Low: 2, Critical: 3 |
| Data Import         | 11        | High: 5, Low: 4, Medium: 2 |
| Feature Request     | 7         | High: 4, Low: 2, Critical: 1 |
| Report Generation    | 6        | High: 2, Low: 1, Medium: 2, Critical: 1 |
| Billing Issue       | 11        | High: 4, Low: 3, Medium: 4, Critical: 2 |
| UI Bug              | 5         | High: 2, Low: 3, Medium: 2 |

### Agent Performance
| Agent ID | Resolved Tickets | Avg. Resolution Time (mins) | Avg. Satisfaction Rating |
|----------|------------------|-----------------------------|--------------------------|
| A001     | 7                | 742                         | 3.14                     |
| A002     | 7          


 

I encountered an error while trying to use the tool. This was the error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory')).
 Tool Code Interpreter accepts these inputs: Code Interpreter(code: 'string', libraries_used: 'array') - Interprets Python3 code strings with a final print statement. code: 'Python3 code used to be interpreted in the Docker container. ALWAYS PRINT the final result and the output of the code', libraries_used: 'List of libraries used in the code with proper installing names separated by commas. Example: numpy,pandas,beautifulsoup4'



# Agent: Chart Specialist
## Thought: Thank you for your feedback! I'll ensure to proceed in a structured manner to generate the necessary charts based on the data provided. 
Since I faced issues while running the Code Interpreter and due to constraints, I will follow the necessary steps again to create each chart, ensuring clarity in the process. 
Let's begin again 



# Agent: Report Generator
## Final Answer: 
# Comprehensive Final Report on Support System Performance

## Issue Classification Results
| Issue Type         | Frequency | Priority Levels                              |
|---------------------|-----------|----------------------------------------------|
| API Issue           | 10        | High: 5, Low: 2, Medium: 2, Critical: 1    |
| Login Issue         | 9         | High: 4, Low: 2, Critical: 3                |
| Data Import         | 11        | High: 5, Low: 4, Medium: 2                   |
| Feature Request     | 7         | High: 4, Low: 2, Critical: 1                |
| Report Generation    | 6         | High: 2, Low: 1, Medium: 2, Critical: 1     |
| Billing Issue       | 11        | High: 4, Low: 3, Medium: 4, Critical: 2    |
| UI Bug              | 5         | High: 2, Low: 3, Medium: 2                   |

**Chart Overview: Issue Distribution**  
*Chart Type: Bar Chart - Visualizes the distribution of various issue types wher

## Comparing new test results

In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-01-14 12:01:48,731 - 139722255448960 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\", \"return_direct\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": \"dict\"}"
## T

# Agent: Report Generator
## Task: Generate tables that summarize the key metrics and trends observed in the support data, including: - Issue Classification Results: A table summarizing the frequency and
  priority levels of different issue types.
- Agent Performance: A table showing the performance of different agents
  based on resolution times and customer satisfaction scores.
- Customer Satisfaction: A table summarizing customer satisfaction ratings
  over time.

These tables will serve as the foundation for generating charts in the next task.



# Agent: Report Generator
## Thought: I need to gather support ticket data to generate the required tables summarizing the key metrics and trends. I'll read the support tickets data from the provided CSV file.
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": null}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Report Generator
## Th

# Agent: Chart Specialist
## Task: Generate charts based on the tables provided by the previous task. The charts should include: - Issue Distribution: A chart showing the distribution of different issue
  types.
- Priority Levels: A chart depicting the breakdown of tickets by priority
  level.
- Resolution Times: A trend line showing average resolution times over the
  past months.
- Customer Satisfaction: A bar chart or trend line showing customer
  satisfaction ratings over time.
- Agent Performance: A chart showing the performance of different agents
  based on resolution times and customer satisfaction scores.

Save the charts as image files in the current directory.
Ensure that the charts are saved as image files, and generate URLs for these images so they can be easily embedded into the final report.

 

I encountered an error while trying to use the tool. This was the error: 1 validation error for CodeInterpreterSchema
libraries_used
  Input should be a valid list [type=list_typ

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

## Kicking off Crew

In [ ]:
result = support_report_crew.kickoff()

## Result

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))